## Imports

In [14]:
import random
from typing import List, Dict, Tuple

## Clases para el algoritmo genetico

In [15]:
class Gene:
    def __init__(self, curso: str, profesor: str, salon: str, dia: str, hora: str, programa: str, creditos: int, semestre: int, estudiante: int):
        self.curso = curso
        self.profesor = profesor
        self.salon = salon
        self.dia = dia
        self.hora = hora
        self.programa = programa
        self.creditos = creditos
        self.semestre = semestre
        self.estudiante = estudiante
        self.violation = False

class Chromosome:
    def __init__(self, genes: List[Gene]):
        self.genes = genes
        self.fitness = 0


In [29]:
class GeneticAlgorithm:
    def __init__(self, cursos: List[Dict], salones: List[str], dias: List[str], horas: List[str], 
                 population_size: int, n_selection:int, mutation_rate: float, crossover_rate: float,
                 salon_props: Dict[str, Dict], fitness_settings: Dict):
        self.cursos = cursos
        self.salones = salones
        self.dias = dias
        self.horas = horas
        self.population_size = population_size
        self.n_selection = n_selection
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.salon_props = salon_props
        self.fitness_settings = fitness_settings
        self.population = []

        self._validate_parameters()

        
    def _validate_parameters(self):
        if len(set(x['curso'] for x in self.cursos)) != len(self.cursos):  # Courses with the same name
            raise ValueError('There are courses with the same name')

        if len(set(self.salones)) != len(self.salones):  # Same classrooms
            raise ValueError('There are salones with the same name')  

        if len(set(self.dias)) != len(self.dias):  # Days more than once
            raise ValueError('There are dias with the same name')

        # parse all the schedules
        self.horas = sorted([self._parse_hour(hour) for hour in self.horas], key=lambda x: (x[0], x[1]))

        # Check for errors in the schedules
        for schedule in self.horas:
            if self._invalid_valid_hour(schedule):
                raise ValueError('Invalid hour')

        for i in range(1, len(self.horas)):
            if self._hours_superpose(self.horas[i-1], self.horas[i]):
                raise ValueError('Hours superpose')

        if (
            self.population_size <= 0 or
            not 0 <= self.n_selection <= self.population_size or
            not 0 <= self.mutation_rate <= 1 or 
            not 0 <= self.crossover_rate <= 1
        ):
            raise ValueError("Invalid model parameters")

    def _invalid_valid_hour(self, schedule):
        """
        This function supposes that every hour is a tupple where the first element is the start hour, 
        the second the start minute the third the end hour and the fourth the end minute.
        """
        # Returns wether one of this conditions is satisfied:
        # - The start hour comes after the end hour
        # - The start hour is the same as the end hour but the start minute is after the end minute
        return schedule[0] > schedule[2] or (schedule[0] == schedule[2] and schedule[1] > schedule[3])

    def _hours_superpose(self, schedule1: str, schedule2: str) -> bool:
        """
        This function supposes that hour1 will always come before hour2.

        Also it supposes that every hour is a tupple where the first element is the start hour, the second the start minute
        the third the end hour and the fourth the end minute.
        """

        # Returns wether one of this conditions is satisfied:
        # - The first schedule's hour ends after the second schedule's hour starts
        # - The first schedule's hour ends at the same time the second schedule's hour starts but the first schedule's
        # hour minutes ends after the second schedule's hour minutes ends
        return (
            schedule1[2] > schedule2[0] or 
            (schedule1[2] == schedule2[0] and schedule1[3] > schedule2[1]) or
            (schedule1[0] == schedule2[0] and schedule1[1] == schedule2[1]  # Same schedules
            and schedule1[2] == schedule2[2] and schedule1[3] == schedule2[3])
        )

    def _parse_hour(self, hour: str) -> tuple[int]:
        def _convert_hour(h: str) -> list[int]:
            res = h.split(".")
            return [res[0], res[1]]

        hours = hour.replace(" ", "").split("-")
        
        return tuple(_convert_hour(hours[0]) + _convert_hour(hours[1]))

    def fitness_function(self, chromosome: Chromosome) -> float:
        penalty = 0

        for gene in chromosome.genes:

            # SalonOverCapacidad
            salon_capacidad = self.salon_props.get(gene.salon, {}).get('capacidad', 0)
            if (self.fitness_settings['salonUsedByOthers']['enable'] and 
                salon_capacidad < gene.estudiante):
                penalty += self.fitness_settings['salonOverCapacidad']['penalty']
              
            # SalonUsedByOthers
            if self.fitness_settings['salonUsedByOthers']['enable']:
                salon_owner = self.salon_props.get(gene.salon, {}).get('owner', ['all'])
                if salon_owner[0] != 'all' and gene.programa not in salon_owner:
                    penalty += self.fitness_settings['salonUsedByOthers']['penalty']
        
        # Check for conflicts
        for i, gene1 in enumerate(chromosome.genes):
            for j, gene2 in enumerate(chromosome.genes[i+1:]): 
                # Condiciones de mismo dia
                if gene1.dia == gene2.dia:
                    
                    # SameProfesorSameDia
                    if (self.fitness_settings['sameProfesorSameDia']['enable'] and
                        gene1.profesor == gene2.profesor):
                        penalty += self.fitness_settings['sameProfesorSameDia']['penalty']

                    # SameProgramaSameSemesterSameDia
                    if (self.fitness_settings['sameProgramaSameSemesterSameDia']['enable'] and
                        gene1.programa == gene2.programa and
                        gene1.semestre == gene2.semestre):
                        penalty += self.fitness_settings['sameProgramaSameSemesterSameDia']['penalty']

                    # SameProfesorHasSequence
                    if (self.fitness_settings['sameProfesorHasSequence']['enable'] and
                        gene1.profesor == gene2.profesor and
                        (self.horas.index(gene1.hora) == self.horas.index(gene2.hora) - 1 or 
                         self.horas.index(gene1.hora) == self.horas.index(gene2.hora) + 1)):
                        penalty += self.fitness_settings['sameProfesorHasSequence']['penalty']

                    # SameProgramaSameSemesterHasSequence
                    if (self.fitness_settings['sameProgramaSameSemesterHasSequence']['enable'] and
                        gene1.programa == gene2.programa and
                        gene1.semestre == gene2.semestre and
                        (self.horas.index(gene1.hora) == self.horas.index(gene2.hora) - 1 or 
                         self.horas.index(gene1.hora) == self.horas.index(gene2.hora) + 1)):
                        penalty += self.fitness_settings['sameProgramaSameSemesterHasSequence']['penalty']

                    # Condiciones de mismo dia y misma hora
                    if gene1.hora == gene2.hora:
                      
                        # SameProfesorSameHora
                        if (self.fitness_settings['sameProfesorSameHora']['enable'] and 
                            gene1.profesor == gene2.profesor):
                            penalty += self.fitness_settings['sameProfesorSameHora']['penalty']
                        
                        # SameProgramaSameSemesterSameHora
                        if (self.fitness_settings['sameProgramaSameSemesterSameHora']['enable'] and 
                            gene1.programa == gene2.programa and 
                            gene1.semestre == gene2.semestre):
                            penalty += self.fitness_settings['sameProgramaSameSemesterSameHora']['penalty']

                        # SameSalonSameHora
                        if (self.fitness_settings['sameSalonSameHora']['enable'] and 
                            gene1.salon == gene2.salon):
                            penalty += self.fitness_settings['sameSalonSameHora']['penalty']
        
        return 1 / (1 + penalty)
    
    def check_violations_in_chromosome(self, chromosome: Chromosome) -> None:
        for gene in chromosome.genes:
            if gene.violation:
              continue

            # SalonOverCapacidad
            salon_capacidad = self.salon_props.get(gene.salon, {}).get('capacidad', 0)
            if (self.fitness_settings['salonUsedByOthers']['enable'] and 
                salon_capacidad < gene.estudiante):
                gene.violation = True
                continue
              
            # SalonUsedByOthers
            if self.fitness_settings['salonUsedByOthers']['enable']:
                salon_owner = self.salon_props.get(gene.salon, {}).get('owner', ['all'])
                if salon_owner[0] != 'all' and gene.programa not in salon_owner:
                    gene.violation = True
        
        # Check for conflicts
        for i, gene1 in enumerate(chromosome.genes):
            if gene1.violation:
                continue
            for j, gene2 in enumerate(chromosome.genes[i+1:]):
                if gene2.violation:
                    continue
                
                # Condiciones de mismo dia
                if gene1.dia == gene2.dia:
                    
                    # SameProfesorSameDia
                    if (self.fitness_settings['sameProfesorSameDia']['enable'] and
                        gene1.profesor == gene2.profesor):
                        gene.violation = True
                        continue

                    # SameProgramaSameSemesterSameDia
                    if (self.fitness_settings['sameProgramaSameSemesterSameDia']['enable'] and
                        gene1.programa == gene2.programa and
                        gene1.semestre == gene2.semestre):
                        gene.violation = True
                        continue


                    # SameProfesorHasSequence
                    if (self.fitness_settings['sameProfesorHasSequence']['enable'] and
                        gene1.profesor == gene2.profesor and
                        (self.horas.index(gene1.hora) == self.horas.index(gene2.hora) - 1 or 
                         self.horas.index(gene1.hora) == self.horas.index(gene2.hora) + 1)):
                        gene.violation = True
                        continue

                    # SameProgramaSameSemesterHasSequence
                    if (self.fitness_settings['sameProgramaSameSemesterHasSequence']['enable'] and
                        gene1.programa == gene2.programa and
                        gene1.semestre == gene2.semestre and
                        (self.horas.index(gene1.hora) == self.horas.index(gene2.hora) - 1 or 
                         self.horas.index(gene1.hora) == self.horas.index(gene2.hora) + 1)):
                        gene.violation = True
                        continue

                    # Condiciones de mismo dia y misma hora
                    if gene1.hora == gene2.hora:
                      
                        # SameProfesorSameHora
                        if (self.fitness_settings['sameProfesorSameHora']['enable'] and 
                            gene1.profesor == gene2.profesor):
                            gene.violation = True
                            continue
                        
                        # SameProgramaSameSemesterSameHora
                        if (self.fitness_settings['sameProgramaSameSemesterSameHora']['enable'] and 
                            gene1.programa == gene2.programa and 
                            gene1.semestre == gene2.semestre):
                            gene.violation = True
                            continue

                        # SameSalonSameHora
                        if (self.fitness_settings['sameSalonSameHora']['enable'] and 
                            gene1.salon == gene2.salon):
                            gene.violation = True

    def evolve(self, generations: int):
        self.initialize_population()
        
        for _ in range(generations):
            new_population = []
            
            while len(new_population) < self.population_size:
                # Selection
                parent1 = self.tournament_selection(self.n_selection)
                parent2 = self.tournament_selection(self.n_selection)
                
                # Crossover
                if random.random() < self.crossover_rate:
                    child1, child2 = self.uniform_crossover(parent1, parent2)
                else:
                    child1 = Chromosome([Gene(g.curso, g.profesor, g.salon, g.dia, g.hora,
                                            g.programa, g.creditos, g.semestre, g.estudiante) for g in parent1.genes])
                    child2 = Chromosome([Gene(g.curso, g.profesor, g.salon, g.dia, g.hora,
                                            g.programa, g.creditos, g.semestre, g.estudiante) for g in parent2.genes])
                
                # Mutation
                if random.random() < self.mutation_rate:
                    self.mutate(child1)
                    self.mutate(child2)
                
                # Fitness Evaluation
                for child in [child1, child2]:
                    child.fitness = self.fitness_function(child)
                  
                # Replacement
                # Solo se quedan los dos mejores de los padres y los hijos a la nueva poblacion
                strongest = [parent1, parent2, child1, child2]
                strongest.sort(key=lambda x: x.fitness, reverse=True)
        
                new_population.extend(strongest[:2])
            
            self.population = sorted(new_population, key=lambda x: x.fitness, reverse=True)[:self.population_size]
        
        best_solution = max(self.population, key=lambda chromo: chromo.fitness)
        top10 = sorted(self.population, key=lambda chromo: chromo.fitness, reverse=True)[:10]
        return best_solution, top10
    
    def initialize_population(self):
        for _ in range(self.population_size):
            chromosome = Chromosome([
                Gene(curso['curso'], curso['profesor'], 
                     random.choice(self.salones), 
                     random.choice(self.dias), 
                     random.choice(self.horas),
                     curso['programa'], curso['creditos'], curso['semestre'], curso['estudiante'])
                for curso in self.cursos
            ])
            self.population.append(chromosome)

    def tournament_selection(self, tournament_size: int) -> Chromosome:
        tournament = random.sample(self.population, tournament_size)
        return max(tournament, key=lambda chromo: chromo.fitness)
    
    def uniform_crossover(self, parent1: Chromosome, parent2: Chromosome) -> Tuple[Chromosome, Chromosome]:
        child1_genes = []
        child2_genes = []
        for gene1, gene2 in zip(parent1.genes, parent2.genes):
            if random.random() < 0.5:
                child1_genes.append(Gene(gene1.curso, gene1.profesor, gene1.salon, gene1.dia, gene1.hora,
                                         gene1.programa, gene1.creditos, gene1.semestre, gene1.estudiante))
                child2_genes.append(Gene(gene2.curso, gene2.profesor, gene2.salon, gene2.dia, gene2.hora,
                                         gene2.programa, gene2.creditos, gene2.semestre, gene2.estudiante))
            else:
                child1_genes.append(Gene(gene2.curso, gene2.profesor, gene2.salon, gene2.dia, gene2.hora,
                                         gene2.programa, gene2.creditos, gene2.semestre, gene2.estudiante))
                child2_genes.append(Gene(gene1.curso, gene1.profesor, gene1.salon, gene1.dia, gene1.hora,
                                         gene1.programa, gene1.creditos, gene1.semestre, gene1.estudiante))
        return Chromosome(child1_genes), Chromosome(child2_genes)
    
    def mutate(self, chromosome: Chromosome):
        self.check_violations_in_chromosome(chromosome)
        for gene in chromosome.genes:
            if gene.violation:
                gene.salon = random.choice(self.salones)
                gene.dia = random.choice(self.dias)
                gene.hora = random.choice(self.horas)

### Input

In [30]:
# dias = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]
# horas = ["08.00 - 08.50", "09.00 - 09.50", "10.00 - 10.50", "11.00 - 11.50", "19.00 - 19.50"]

# salones = ["R.1&2", "R.3", "R.4", "R.5", "R.6", "R.7", "R.8", "R.9&10"]
# salon_props = {
#     "R.1&2": {"owner": ["Derecho", "Derecho (AM)", "Administracion", "Contabilidad"], "capacidad": 150},
#     "R.3": {"owner": ["all"], "capacidad": 40},
#     "R.4": {"owner": ["all"], "capacidad": 40},
#     "R.5": {"owner": ["all"], "capacidad": 40},
#     "R.6": {"owner": ["all"], "capacidad": 40},
#     "R.7": {"owner": ["all"], "capacidad": 40},
#     "R.8": {"owner": ["all"], "capacidad": 40},
#     "R.9&10": {"owner": ["all"], "capacidad": 80}
# }

# Para pruebas pequeñas
dias = ["Lunes", "Martes"]
horas = ["08.00 - 08.50", "09.00 - 09.50", "10.00 - 10.50", "11.00 - 11.50"]
salones = ["R.1&2", "R.3", "R.4"]
salon_props = {
    "R.1&2": {"owner": ["Administracion", "Contabilidad"], "capacidad": 150},
    "R.3": {"owner": ["Derecho"], "capacidad": 40},
    "R.4": {"owner": ["Ing Sistemas"], "capacidad": 100}
}

cursos = [
    {"programa": "Administracion", "curso": "Introduccion a la microeconomía", "estudiante": 30,
     "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la macroeconomía", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la contabilidad", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Administracion", "curso": "Introduccion a la administracion", "estudiante": 30,
    "profesor": "Duvan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Derecho", "curso": "Introduccion a la estadistica", "estudiante": 30,
    "profesor": "Johan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Ing Sistemas", "curso": "Introduccion a la programacion", "estudiante": 50,
    "profesor": "Juanse", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Derecho", "curso": "Introduccion a la matematica", "estudiante": 100,
    "profesor": "Johan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Ing Sistemas", "curso": "Introduccion a la fisica", "estudiante": 100,
    "profesor": "Johan", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Ing Sistemas", "curso": "Introduccion a la quimica", "estudiante": 30,
    "profesor": "Juanse", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Ing Sistemas", "curso": "Introduccion al dolor", "estudiante": 110,
    "profesor": "Isis", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Ing Sistemas", "curso": "Introduccion a la vida", "estudiante": 55,
    "profesor": "Isis", "creditos": 3, "semestre": 1, "required": True},
    {"programa": "Medicina", "curso": "Introduccion a la muerte", "estudiante": 1,
    "profesor": "Isis", "creditos": 3, "semestre": 1, "required": True},
]

ga_params = {
    "nPopulations": 1000,
    "npops": 10,
    "nSelection": 3,
    "pCrossover": 0.85,
    "pMutation": 0.14,
    "fitnessThresshold": 0.8,
    "nSolution": 1,
    "stoppingCondition": {"nFitnessNoChange": 1000, "fitnessMax": 1}
}

fitness_settings = {
    "sameProfesorSameHora": {"enable": True, "penalty": 1},
    "sameProgramaSameSemesterSameHora": {"enable": True, "penalty": 1},
    "sameSalonSameHora": {"enable": True, "penalty": 1},
    "horaOver": {"enable": False, "penalty": 1}, # A pesar de la existencia de esta condicion en el problema original, la solucion que proponemos no la tiene en cuenta
    "salonOverCapacidad": {"enable": True, "penalty": 1},
    "salonUsedByOthers": {"enable": True, "penalty": 0.01},
    "sameProfesorSameDia": {"enable": True, "penalty": 0.001},
    "sameProfesorHasSequence": {"enable": True, "penalty": 0.001},
    "sameProgramaSameSemesterSameDia": {"enable": True, "penalty": 0.001},
    "sameProgramaSameSemesterHasSequence": {"enable": True, "penalty": 0.001}
}

## Solucionar

In [31]:
ga = GeneticAlgorithm(
    cursos=cursos,                              # Cursos a programar
    salones=salones,                            # Salones disponibles
    dias=dias,                                  # Dias disponibles
    horas=horas,                                # Horas disponibles
    population_size=ga_params["nPopulations"],  # Tamaño de la poblacion
    n_selection=ga_params["nSelection"],        # Tamaño del torneo
    mutation_rate=ga_params["pMutation"],       # Probabilidad de mutacion
    crossover_rate=ga_params["pCrossover"],     # Probabilidad de crossover
    salon_props=salon_props,                    # Propiedades de los salones
    fitness_settings=fitness_settings           # Configuracion de las penalizaciones
)

best_solution, top10 = ga.evolve(generations=ga_params["stoppingCondition"]["nFitnessNoChange"])

# Output
print("\nBest Solution:")
print(f"Fitness: {best_solution.fitness:.4f}")
print("Schedule:")
for i, gene in enumerate(best_solution.genes, 1):
    print(f"{i}. curso: {gene.curso}")
    print(f"   profesor: {gene.profesor}")
    print(f"   salon: {gene.salon}")
    print(f"   dia: {gene.dia}")
    print(f"   hora: {gene.hora}")
    print(f"   programa: {gene.programa}")
    print(f"   creditos: {gene.creditos}")
    print(f"   Semester: {gene.semestre}")
    print(f"   estudiantes: {gene.estudiante}")
    print()

for i, solution in enumerate(top10, 1):
    print(f"\nTop {i} Solution:")
    print(f"Fitness: {solution.fitness:.4f}")
    print("Schedule:")
    for i, gene in enumerate(solution.genes, 1):
        print(f"{i}. curso: {gene.curso}")
        print(f"   profesor: {gene.profesor}")
        print(f"   salon: {gene.salon}")
        print(f"   dia: {gene.dia}")
        print(f"   hora: {gene.hora}")
        print(f"   programa: {gene.programa}")
        print(f"   creditos: {gene.creditos}")
        print(f"   Semester: {gene.semestre}")
        print(f"   estudiantes: {gene.estudiante}")
        print()

['08.00 - 08.50', '09.00 - 09.50', '10.00 - 10.50', '11.00 - 11.50']

Best Solution:
Fitness: 0.9606
Schedule:
1. curso: Introduccion a la microeconomía
   profesor: Duvan
   salon: R.1&2
   dia: Martes
   hora: ('11', '00', '11', '50')
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

2. curso: Introduccion a la macroeconomía
   profesor: Duvan
   salon: R.1&2
   dia: Lunes
   hora: ('11', '00', '11', '50')
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

3. curso: Introduccion a la contabilidad
   profesor: Duvan
   salon: R.1&2
   dia: Martes
   hora: ('08', '00', '08', '50')
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

4. curso: Introduccion a la administracion
   profesor: Duvan
   salon: R.1&2
   dia: Lunes
   hora: ('09', '00', '09', '50')
   programa: Administracion
   creditos: 3
   Semester: 1
   estudiantes: 30

5. curso: Introduccion a la estadistica
   profesor: Johan
   salon: R.3
  